In [1]:
import json
import pathpy as pp
from utils import Node
import pickle

In [2]:
with open("data/node_tree.pickle", "rb") as f:
    tree = pickle.load(f)

In [3]:
def clean_name(name):
    out = name.replace('\n', '')
    out = out.split('[')[0]
    out = out.split('(')[0]
    out = out.strip()
    return out

In [4]:
def compat_name(directory, name):
    n_try = name
    n = 2
    if name not in directory:
        return name
    else:
        while n_try in directory:
            n_try = name + str(n)
            n += 1
        return n_try

compat_name(['a', 'b', 'b1'], 'b')

'b2'

In [49]:
def walk(node, alias=None, n=pp.Network(directed=False), depth=0):
    e1 = alias or node.name
    children = node.children
    if e1 not in list(n.nodes.keys()):
        n.add_node(e1, depth=depth)
    for child in children:
        # Names must be new (hence function compat_name)
        # otherwise new node is not added, and subsequent children are
        # appended in the wrong place
        e2_comp = compat_name(n.nodes.keys(), clean_name(child.name))
        n.add_node(e2_comp, depth=depth, is_terminal=child.is_terminal)
        n.add_edge(e1, e2_comp)
        assert n is not None
        n = walk(child, alias=e2_comp, n=n, depth=depth+1)
    
    return n

network = walk

n = network(tree['uralic'], n=pp.Network())

In [45]:
nodes = list(n.nodes.keys())
edges = list(n.edges)
len(nodes)

598

In [40]:
res = []
search = 'Northwestern'
for el in edges:
    if search in el[0] or search in el[1]:
        res.append(el)
res

[('Northwestern', 'Outer Languages'),
 ('Dardic', 'Northwestern'),
 ('Northwestern', 'Sindhi'),
 ('Northwestern2', 'Western4'),
 ('Balochi', 'Northwestern2'),
 ('Caspian', 'Northwestern2'),
 ('Central Iran', 'Northwestern2'),
 ('Central Iran', 'Fars, Northwestern'),
 ('Kurdish', 'Northwestern2'),
 ('Northwestern2', 'Ormuri-Parachi'),
 ('Northwestern2', 'Semnani'),
 ('Northwestern2', 'Talysh'),
 ('Northwestern2', 'Unclassified7'),
 ('Northwestern2', 'Zaza-Gorani')]

In [48]:
tt = pp.Network()
tt.add_node('a')
tt.add_node('a')
len(tt)

TypeError: object of type 'Network' has no len()

In [50]:
terminal = []

for node, pars in n.nodes.items():
    if pars.get('is_terminal') is True:
        terminal.append(node)

pp.visualisation.plot(n,
                      height=1000,
                      width=1000,
                      node_color = {**{el: "#FF6699" for el in terminal}, **{"Uralic": "#000000"}},
                      node_text = {el: ''.join([c for c in el if not c.isdigit()]) for el in n.nodes},
                      force_alpha=0.2, force_repel=-10)

In [17]:
len(n.nodes)

598

In [67]:
dd.get("Kashmiri2")

'Kashmiri'

In [56]:
n.nodes.items()

dict_items([('Indo-European', {'depth': 0, 'inweight': 8.0, 'outweight': 8.0, 'degree': 8}), ('Albanian', {'depth': 0, 'is_terminal': False, 'inweight': 3.0, 'outweight': 3.0, 'degree': 3}), ('Gheg', {'depth': 1, 'is_terminal': False, 'inweight': 2.0, 'outweight': 2.0, 'degree': 2}), ('Albanian, Gheg', {'depth': 2, 'is_terminal': True, 'inweight': 1.0, 'outweight': 1.0, 'degree': 1}), ('Tosk', {'depth': 1, 'is_terminal': False, 'inweight': 4.0, 'outweight': 4.0, 'degree': 4}), ('Albanian, Arbëreshë', {'depth': 2, 'is_terminal': True, 'inweight': 1.0, 'outweight': 1.0, 'degree': 1}), ('Albanian, Arvanitika', {'depth': 2, 'is_terminal': True, 'inweight': 1.0, 'outweight': 1.0, 'degree': 1}), ('Albanian, Tosk', {'depth': 2, 'is_terminal': True, 'inweight': 1.0, 'outweight': 1.0, 'degree': 1}), ('Armenian', {'depth': 0, 'is_terminal': False, 'inweight': 3.0, 'outweight': 3.0, 'degree': 3}), ('Armenian2', {'depth': 1, 'is_terminal': True, 'inweight': 1.0, 'outweight': 1.0, 'degree': 1}), ('

In [57]:
red

['Albanian, Gheg',
 'Albanian, Arbëreshë',
 'Albanian, Arvanitika',
 'Albanian, Tosk',
 'Armenian2',
 'Armenian, Western',
 'Latgalian',
 'Latvian, Standard',
 'Lithuanian',
 'Samogitian',
 'Prussian',
 'Belarusian',
 'Russian',
 'Rusyn',
 'Ukrainian',
 'Bulgarian',
 'Macedonian',
 'Slavonic, Church',
 'Bosnian',
 'Chakavian',
 'Croatian',
 'Montenegrin',
 'Serbian',
 'Slavomolisano',
 'Slovene',
 'Czech',
 'Slovak',
 'Kashubian',
 'Polish',
 'Silesian',
 'Sorbian, Lower',
 'Sorbian, Upper',
 'Breton',
 'Cornish',
 'Welsh',
 'Irish',
 'Manx',
 'Scottish Gaelic',
 'Övdalian',
 'Norwegian',
 'Danish2',
 'Swedish2',
 'Faroese',
 'Icelandic',
 'English2',
 'Scots',
 'Frisian2',
 'Frisian, Northern',
 'Saterfriesisch',
 'Hunsrik',
 'German, Standard',
 'Saxon, Upper',
 'Silesian, Lower',
 'Wymysorys',
 'German, Pennsylvania',
 'Palatinate Franconian',
 'Ripuarian',
 'Luxembourgish',
 'Eastern Franconian',
 'German, Colonia Tovar',
 'German, Swiss',
 'Swabian',
 'Walser',
 'Bavarian',
 'Cimb

In [ ]:
pp.visualisation.plot?

In [ ]:
sorted(list(n.nodes.keys()))

In [ ]:
n.nodes['Catalan']

In [ ]:
n.nodes['Unclassified']

In [ ]:
a = pp.Network()
a.add_node('a')
a.add_node('a')
a.nodes

In [ ]:
n.nodes['Catalan']

In [ ]:
n.nodes['indo-european']

In [ ]:
indo[1][0]

In [ ]:
families[1][2][1][0]

In [ ]:

children = families[1]
for child in children:
    for cchild in child[1]:
        print(cchild[0])


In [ ]:
n

In [ ]:
q

In [ ]:
n = pp.Network()
parent = n.add_node('LANGUAGES')
for a, b in families:
    n.add_edge(a, 'LANGUAGES')
    for aa, bb in b:
        n.add_edge(a, aa)
n

In [ ]:
a = dict(a=1, b=2)
list(a.items())

In [ ]:
n = pp.Network()
n.add_node('a')
n.add_edge('a', 'b')
n